In [1]:
from IPython.display import IFrame, Image

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "valeria.lupanova Spark RDD app") 

sc = SparkContext(conf=conf)

In [4]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/data/home/valeria.lupanova/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.11-2.4.5.jar,/data/home/valeria.lupanova/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,/data/home/valeria.lupanova/.ivy2/jars/databricks_spark-sklearn-0.2.3.jar,/data/home/valeria.lupanova/.ivy2/jars/org.apache.kafka_kafka-clients-2.0.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.lz4_lz4-java-1.4.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.7.3.jar,/data/home/valeria.lupanova/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.history.fs.cleaner.interval', '7d'),
 ('spark.shuffle.io.serverThread

In [5]:
k_ = 50
ci_ = [2.576, 0.99]

In [6]:
!hdfs dfs -ls /user/valeria.lupanova/lab06/ml-100k

Found 23 items
-rw-r--r--   2 valeria.lupanova valeria.lupanova       6403 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/README
-rw-r--r--   2 valeria.lupanova valeria.lupanova        716 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/allbut.pl
-rw-r--r--   2 valeria.lupanova valeria.lupanova        643 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/mku.sh
-rw-r--r--   2 valeria.lupanova valeria.lupanova    1979173 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.data
-rw-r--r--   2 valeria.lupanova valeria.lupanova        202 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.genre
-rw-r--r--   2 valeria.lupanova valeria.lupanova         36 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.info
-rw-r--r--   2 valeria.lupanova valeria.lupanova     236344 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.item
-rw-r--r--   2 valeria.lupanova valeria.lupanova        193 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.occupation
-rw-r--

In [7]:
u_item_ = sc.textFile("/user/valeria.lupanova/lab06/ml-100k/u.item")
u_item_

/user/valeria.lupanova/lab06/ml-100k/u.item MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [8]:
u_item_.take(5)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']

In [9]:
u_item_ = u_item_.map(lambda x: x.split("|"))

In [10]:
u_item_.take(5)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2',
  'GoldenEye (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?GoldenEye%20(1995)',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0'],
 ['3',
  'Four Rooms (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0'],
 ['4',
  'Get Shorty (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['5',
  'Copycat 

In [11]:
u_data_ = sc.textFile("/user/valeria.lupanova/lab06/ml-100k/u.data")
u_data_

/user/valeria.lupanova/lab06/ml-100k/u.data MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
u_data_.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [13]:
u_data_ = u_data_.map(lambda x: x.split("\t"))

In [14]:
u_data_.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [15]:
type(u_data_)

pyspark.rdd.PipelinedRDD

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
hasattr(u_data_, "toDF")

True

In [17]:
di = u_item_.map(lambda x : (x[0], x[1]))\
            .toDF(("ItemID", "ItemName"))

In [18]:
di.select('ItemID', 'ItemName').orderBy('ItemName', ascending=True).show()

+------+--------------------+
|ItemID|            ItemName|
+------+--------------------+
|  1300|'Til There Was Yo...|
|  1353|        1-900 (1994)|
|   225|101 Dalmatians (1...|
|   178| 12 Angry Men (1957)|
|   330|          187 (1997)|
|  1011|2 Days in the Val...|
|   141|20,000 Leagues Un...|
|   135|2001: A Space Ody...|
|   314|3 Ninjas: High No...|
|   615|39 Steps, The (1935)|
|  1021|        8 1/2 (1963)|
|  1664|8 Heads in a Duff...|
|  1443|    8 Seconds (1994)|
|  1251|A Chef in Love (1...|
|  1440|Above the Rim (1994)|
|   291|Absolute Power (1...|
|   164|   Abyss, The (1989)|
|    67|Ace Ventura: Pet ...|
|   364|Ace Ventura: When...|
|   757|Across the Sea of...|
+------+--------------------+
only showing top 20 rows



In [19]:
import re 
from pyspark.sql import functions as fsql


di = di.selectExpr("ItemID", "regexp_replace(ItemName,'[^0-9a-zA-Z ]+','') AS ItemName")
di.show()

+------+--------------------+
|ItemID|            ItemName|
+------+--------------------+
|     1|      Toy Story 1995|
|     2|      GoldenEye 1995|
|     3|     Four Rooms 1995|
|     4|     Get Shorty 1995|
|     5|        Copycat 1995|
|     6|Shanghai Triad Ya...|
|     7| Twelve Monkeys 1995|
|     8|           Babe 1995|
|     9|Dead Man Walking ...|
|    10|    Richard III 1995|
|    11|    Seven Se7en 1995|
|    12|Usual Suspects Th...|
|    13|Mighty Aphrodite ...|
|    14|     Postino Il 1994|
|    15|Mr Hollands Opus ...|
|    16|French Twist Gazo...|
|    17|From Dusk Till Da...|
|    18|White Balloon The...|
|    19|  Antonias Line 1995|
|    20|Angels and Insect...|
+------+--------------------+
only showing top 20 rows



In [20]:
df = u_data_.map(lambda x : (x[0], x[1], x[2]))\
            .toDF(("UserID", "ItemID", "Score"))

In [21]:
df.show()

+------+------+-----+
|UserID|ItemID|Score|
+------+------+-----+
|   196|   242|    3|
|   186|   302|    3|
|    22|   377|    1|
|   244|    51|    2|
|   166|   346|    1|
|   298|   474|    4|
|   115|   265|    2|
|   253|   465|    5|
|   305|   451|    3|
|     6|    86|    3|
|    62|   257|    2|
|   286|  1014|    5|
|   200|   222|    5|
|   210|    40|    3|
|   224|    29|    3|
|   303|   785|    3|
|   122|   387|    5|
|   194|   274|    2|
|   291|  1042|    4|
|   234|  1184|    2|
+------+------+-----+
only showing top 20 rows



In [22]:
df.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- ItemID: string (nullable = true)
 |-- Score: string (nullable = true)



In [23]:
from pyspark.sql.types import IntegerType

df = df.withColumn("ScoreInt", df["Score"].cast(IntegerType())).drop("Score")
df.show()

+------+------+--------+
|UserID|ItemID|ScoreInt|
+------+------+--------+
|   196|   242|       3|
|   186|   302|       3|
|    22|   377|       1|
|   244|    51|       2|
|   166|   346|       1|
|   298|   474|       4|
|   115|   265|       2|
|   253|   465|       5|
|   305|   451|       3|
|     6|    86|       3|
|    62|   257|       2|
|   286|  1014|       5|
|   200|   222|       5|
|   210|    40|       3|
|   224|    29|       3|
|   303|   785|       3|
|   122|   387|       5|
|   194|   274|       2|
|   291|  1042|       4|
|   234|  1184|       2|
+------+------+--------+
only showing top 20 rows



In [24]:
#1 Количество человек n, поставивших рейтинг фильму 

df_1_ = df.groupBy("ItemID").agg(fsql.sum("ScoreInt").alias("sum_Score"), \
                               fsql.count("UserID").alias("count_ItemID"))
df_1_.show()

+------+---------+------------+
|ItemID|sum_Score|count_ItemID|
+------+---------+------------+
|   829|       90|          34|
|  1436|        5|           2|
|   467|      182|          48|
|   691|       56|          16|
|  1090|       89|          37|
|   675|      192|          54|
|   296|       20|           6|
|  1512|       24|           6|
|  1159|       39|          11|
|  1572|        1|           1|
|   451|      569|         170|
|   944|      129|          43|
|   125|      868|         244|
|   853|       54|          14|
|  1394|       17|           7|
|  1372|        9|           3|
|   800|       75|          26|
|  1669|        2|           1|
|   919|      364|          96|
|   870|       25|           9|
+------+---------+------------+
only showing top 20 rows



In [25]:
#2 Средний рейтинг фильма (сумма рейтингов фильма / количество человек, оценивших фильм)

df_1_ = df_1_.withColumn("avg_Rating_2", fsql.col("sum_Score")/fsql.col("count_ItemID"))
df_1_.show()

+------+---------+------------+------------------+
|ItemID|sum_Score|count_ItemID|      avg_Rating_2|
+------+---------+------------+------------------+
|   829|       90|          34|2.6470588235294117|
|  1436|        5|           2|               2.5|
|   467|      182|          48|3.7916666666666665|
|   691|       56|          16|               3.5|
|  1090|       89|          37|2.4054054054054053|
|   675|      192|          54|3.5555555555555554|
|   296|       20|           6|3.3333333333333335|
|  1512|       24|           6|               4.0|
|  1159|       39|          11|3.5454545454545454|
|  1572|        1|           1|               1.0|
|   451|      569|         170| 3.347058823529412|
|   944|      129|          43|               3.0|
|   125|      868|         244| 3.557377049180328|
|   853|       54|          14| 3.857142857142857|
|  1394|       17|           7|2.4285714285714284|
|  1372|        9|           3|               3.0|
|   800|       75|          26|

In [26]:
#3 Количество человек m, оценивших фильм положительно. Оценки 4 и выше считаются положительными

df_2_ = df.groupBy("ItemID").agg(fsql.sum(fsql.when(fsql.col("ScoreInt") >= 4, 1).otherwise(0)).alias("count_pos_avg_Rating"))
df_2_.show()

+------+--------------------+
|ItemID|count_pos_avg_Rating|
+------+--------------------+
|   829|                   7|
|  1436|                   0|
|   467|                  31|
|   691|                   9|
|  1090|                   6|
|   675|                  30|
|   296|                   3|
|  1512|                   4|
|  1159|                   6|
|  1572|                   0|
|   451|                  87|
|   944|                  14|
|   125|                 125|
|   853|                  11|
|  1394|                   3|
|  1372|                   2|
|   800|                   9|
|  1669|                   0|
|   919|                  63|
|   870|                   1|
+------+--------------------+
only showing top 20 rows



In [27]:
#4 Доля людей, оценивших фильм положительно (пункт 3 / пункт 1 или m / n)

df_4_ = df_2_.join(df_1_, df_2_.ItemID == df_1_.ItemID).drop(df_2_.ItemID)
df_4_.show()

+--------------------+------+---------+------------+------------------+
|count_pos_avg_Rating|ItemID|sum_Score|count_ItemID|      avg_Rating_2|
+--------------------+------+---------+------------+------------------+
|                   6|  1090|       89|          37|2.4054054054054053|
|                   6|  1159|       39|          11|3.5454545454545454|
|                   0|  1436|        5|           2|               2.5|
|                   4|  1512|       24|           6|               4.0|
|                   0|  1572|        1|           1|               1.0|
|                   3|   296|       20|           6|3.3333333333333335|
|                  31|   467|      182|          48|3.7916666666666665|
|                  30|   675|      192|          54|3.5555555555555554|
|                   9|   691|       56|          16|               3.5|
|                   7|   829|       90|          34|2.6470588235294117|
|                 125|   125|      868|         244| 3.557377049

In [28]:
df_4_ = df_4_.withColumn("prt_pos_Rating", fsql.col("count_pos_avg_Rating")/fsql.col("count_ItemID"))
df_4_.show()

+--------------------+------+---------+------------+------------------+-------------------+
|count_pos_avg_Rating|ItemID|sum_Score|count_ItemID|      avg_Rating_2|     prt_pos_Rating|
+--------------------+------+---------+------------+------------------+-------------------+
|                   6|  1090|       89|          37|2.4054054054054053|0.16216216216216217|
|                   6|  1159|       39|          11|3.5454545454545454| 0.5454545454545454|
|                   0|  1436|        5|           2|               2.5|                0.0|
|                   4|  1512|       24|           6|               4.0| 0.6666666666666666|
|                   0|  1572|        1|           1|               1.0|                0.0|
|                   3|   296|       20|           6|3.3333333333333335|                0.5|
|                  31|   467|      182|          48|3.7916666666666665| 0.6458333333333334|
|                  30|   675|      192|          54|3.5555555555555554| 0.555555

In [29]:
#5 Глобальное среднее по всему датасету. Сумма всех оценок по всем фильмам /Количество всех оценок по всем фильмам

mu_ = df_4_.select(fsql.sum("sum_Score")).collect()[0][0]/ \
          df_4_.select(fsql.sum("count_ItemID")).collect()[0][0]
mu_

3.52986

In [30]:
#6 Оценка, поправленная на нехватку данных

df_6_ = df_1_.withColumn("avg_Rating_6", (fsql.col("sum_Score")+(k_*mu_))/(fsql.col("count_ItemID")+k_))
df_6_.show()

+------+---------+------------+------------------+------------------+
|ItemID|sum_Score|count_ItemID|      avg_Rating_2|      avg_Rating_6|
+------+---------+------------+------------------+------------------+
|   829|       90|          34|2.6470588235294117| 3.172535714285715|
|  1436|        5|           2|               2.5|3.4902500000000005|
|   467|      182|          48|3.7916666666666665|3.6580918367346946|
|   691|       56|          16|               3.5|3.5226212121212126|
|  1090|       89|          37|2.4054054054054053|  3.05164367816092|
|   675|      192|          54|3.5555555555555554|3.5432019230769236|
|   296|       20|           6|3.3333333333333335| 3.508803571428572|
|  1512|       24|           6|               4.0| 3.580232142857143|
|  1159|       39|          11|3.5454545454545454|3.5326721311475415|
|  1572|        1|           1|               1.0|3.4802549019607847|
|   451|      569|         170| 3.347058823529412|3.3886045454545455|
|   944|      129|  

In [31]:
#7 Нижнюю и верхнюю границы доверительного 
# интервала оценки (Wilson score interval)
# из лекции с заданным уровнем доверия

df_4_.show()

+--------------------+------+---------+------------+------------------+-------------------+
|count_pos_avg_Rating|ItemID|sum_Score|count_ItemID|      avg_Rating_2|     prt_pos_Rating|
+--------------------+------+---------+------------+------------------+-------------------+
|                   6|  1090|       89|          37|2.4054054054054053|0.16216216216216217|
|                   6|  1159|       39|          11|3.5454545454545454| 0.5454545454545454|
|                   0|  1436|        5|           2|               2.5|                0.0|
|                   4|  1512|       24|           6|               4.0| 0.6666666666666666|
|                   0|  1572|        1|           1|               1.0|                0.0|
|                   3|   296|       20|           6|3.3333333333333335|                0.5|
|                  31|   467|      182|          48|3.7916666666666665| 0.6458333333333334|
|                  30|   675|      192|          54|3.5555555555555554| 0.555555

In [32]:
import math  
from scipy.special import ndtri

df_7_ = df_4_.withColumn("sqrt_", pow((1/fsql.col("count_ItemID"))*\
                         (fsql.col("prt_pos_Rating")*(1-fsql.col("prt_pos_Rating")))+\
                         ((1/(4*pow(fsql.col("count_ItemID"),2)))*pow(ci_[0],2)),0.5)*ci_[0])

df_7_.show()


+--------------------+------+---------+------------+------------------+-------------------+-------------------+
|count_pos_avg_Rating|ItemID|sum_Score|count_ItemID|      avg_Rating_2|     prt_pos_Rating|              sqrt_|
+--------------------+------+---------+------------+------------------+-------------------+-------------------+
|                   6|  1090|       89|          37|2.4054054054054053|0.16216216216216217|0.18002233056404485|
|                   6|  1159|       39|          11|3.5454545454545454| 0.5454545454545454| 0.4904539277916316|
|                   0|  1436|        5|           2|               2.5|                0.0| 1.6589440000000002|
|                   4|  1512|       24|           6|               4.0| 0.6666666666666666| 0.7426693991922602|
|                   0|  1572|        1|           1|               1.0|                0.0| 3.3178880000000004|
|                   3|   296|       20|           6|3.3333333333333335|                0.5| 0.7630720946

In [33]:
df_7_ = df_7_.withColumn("WS_interval_up", fsql.col("prt_pos_Rating")+\
                         (pow(ci_[0],2)*(1/(2*fsql.col("count_ItemID"))))\
                        +fsql.col("sqrt_"))

df_7_ = df_7_.withColumn("WS_interval_bottom", fsql.col("prt_pos_Rating")+\
                         (pow(ci_[0],2)*(1/(2*fsql.col("count_ItemID"))))\
                        -fsql.col("sqrt_"))

df_7_ = df_7_.withColumn("r_WS_", (1/(1+((1/fsql.col("count_ItemID"))*pow(ci_[0],2)))))

df_7_ = df_7_.withColumn("r_WS_interval_up", fsql.col("r_WS_")*fsql.col("WS_interval_up"))

df_7_ = df_7_.withColumn("r_WS_interval_bottom", fsql.col("r_WS_")*fsql.col("WS_interval_bottom"))

df_t_ = df_7_.select(fsql.col("ItemID"),\
                     fsql.col("r_WS_"),\
                     fsql.col("WS_interval_up"),\
                     fsql.col("WS_interval_bottom"),\
                     fsql.col("r_WS_interval_up"),\
                     fsql.col("r_WS_interval_bottom"))

In [34]:
df_t_.show()

+------+-------------------+-------------------+-------------------+------------------+--------------------+
|ItemID|              r_WS_|     WS_interval_up| WS_interval_bottom|  r_WS_interval_up|r_WS_interval_bottom|
+------+-------------------+-------------------+-------------------+------------------+--------------------+
|  1090| 0.8479280854315505|0.43185714137485565|0.07181248024676598|0.3661837990659238|0.060891818885731315|
|  1159| 0.6237321227033049| 1.3375346550643588| 0.3566267994810956|0.8342633295925252| 0.22243959065322963|
|  1436|0.23159470555975514| 3.3178880000000004|                0.0| 0.768405294440245|                 0.0|
|  1512|0.47484222575645535|   1.96231739919226| 0.4769786008077397|0.9317911614730714| 0.22648958044574694|
|  1572|0.13096245882540294|  6.635776000000001|                0.0|0.8690375411745972|                 0.0|
|   296|0.47484222575645535| 1.8160534280149419| 0.2899092386517248|0.8623388518512557|  0.1376611481487444|
|   467| 0.87854522

In [44]:
df_8_ = df_7_.select(fsql.col("ItemID").alias("ItemID"),\
                     fsql.col("count_ItemID").alias("top10_rates"),\
                     fsql.col("r_WS_interval_bottom").alias("top10_lower"))

df_8_ = df_8_.join(df_1_.withColumnRenamed('avg_Rating_2', 'top10_average'), \
                   df_8_.ItemID == df_1_.ItemID).drop(df_1_.ItemID).drop(df_1_.sum_Score).drop(df_1_.count_ItemID)

df_8_ = df_8_.join(df_6_.withColumnRenamed('avg_Rating_6', 'top10_rating'), \
                   df_8_.ItemID == df_6_.ItemID).drop(df_6_.ItemID)\
                                                .drop(df_6_.sum_Score)\
                                                .drop(df_6_.count_ItemID)\
                                                .drop(df_6_.avg_Rating_2)

df_8_ = df_8_.join(di, df_8_.ItemID == di.ItemID).drop(di.ItemID)

df_8_.show()

+------+-----------+--------------------+------------------+------------------+--------------------+
|ItemID|top10_rates|         top10_lower|     top10_average|      top10_rating|            ItemName|
+------+-----------+--------------------+------------------+------------------+--------------------+
|  1090|         37|0.060891818885731315|2.4054054054054053|  3.05164367816092|         Sliver 1993|
|  1159|         11| 0.22243959065322963|3.5454545454545454|3.5326721311475415|        Stalker 1979|
|  1436|          2|                 0.0|               2.5|3.4902500000000005|       Mr Jones 1993|
|  1512|          6| 0.22648958044574694|               4.0| 3.580232142857143|World of Apu The ...|
|  1572|          1|                 0.0|               1.0|3.4802549019607847|Wend Kuuni Gods G...|
|   296|          6|  0.1376611481487444|3.3333333333333335| 3.508803571428572|    Promesse La 1996|
|   467|         48| 0.46050735991107683|3.7916666666666665|3.6580918367346946|   Bronx Tal

In [45]:
df_8_.select('ItemID', 'ItemName', 'top10_rates').orderBy('top10_rates', ascending=False).show(10)

+------+--------------------+-----------+
|ItemID|            ItemName|top10_rates|
+------+--------------------+-----------+
|    50|      Star Wars 1977|        583|
|   258|        Contact 1997|        509|
|   100|          Fargo 1996|        508|
|   181|Return of the Jed...|        507|
|   294|      Liar Liar 1997|        485|
|   286|English Patient T...|        481|
|   288|         Scream 1996|        478|
|     1|      Toy Story 1995|        452|
|   300|  Air Force One 1997|        431|
|   121|Independence Day ...|        429|
+------+--------------------+-----------+
only showing top 10 rows



In [46]:
top10_rates = df_8_.select('ItemID').orderBy('top10_rates', ascending=False)
top10_rates_ = [row.ItemID for row in top10_rates.toLocalIterator()]
top10_rates_ = top10_rates_[0:10]
top10_rates_ = list(map(int, top10_rates_))
top10_rates_

[50, 258, 100, 181, 294, 286, 288, 1, 300, 121]

In [47]:
df_8_.select('ItemID', 'ItemName', 'top10_lower').orderBy('top10_lower', ascending=False).show(10)

+------+--------------------+------------------+
|ItemID|            ItemName|       top10_lower|
+------+--------------------+------------------+
|    64|Shawshank Redempt...|0.8457526835447277|
|   318|Schindlers List 1993|0.8336980984637925|
|    98|Silence of the La...|0.8334527970235317|
|   479|        Vertigo 1958|0.8332601854479507|
|   483|     Casablanca 1942|0.8262807973523965|
|    50|      Star Wars 1977|0.8182013061987115|
|   603|    Rear Window 1954|0.8161443229080241|
|   427|To Kill a Mocking...|0.8134728086457121|
|   357|One Flew Over the...| 0.808875556038692|
|    12|Usual Suspects Th...|0.8066550869388946|
+------+--------------------+------------------+
only showing top 10 rows



In [48]:
from pyspark.sql.window import Window

df_9_ = df_8_.select(fsql.col("ItemID").alias("ItemID"),\
                     fsql.col("ItemName").alias("ItemName"),\
                     fsql.col("top10_lower").alias("top10_lower"),\
                     fsql.row_number().over(Window.partitionBy()\
                                            .orderBy(fsql.col("top10_lower").desc(),\
                                                     fsql.col("ItemName").asc())).alias("rn"))
df_9_.show()

+------+--------------------+------------------+---+
|ItemID|            ItemName|       top10_lower| rn|
+------+--------------------+------------------+---+
|    64|Shawshank Redempt...|0.8457526835447277|  1|
|   318|Schindlers List 1993|0.8336980984637925|  2|
|    98|Silence of the La...|0.8334527970235317|  3|
|   479|        Vertigo 1958|0.8332601854479507|  4|
|   483|     Casablanca 1942|0.8262807973523965|  5|
|    50|      Star Wars 1977|0.8182013061987115|  6|
|   603|    Rear Window 1954|0.8161443229080241|  7|
|   427|To Kill a Mocking...|0.8134728086457121|  8|
|   357|One Flew Over the...| 0.808875556038692|  9|
|    12|Usual Suspects Th...|0.8066550869388946| 10|
|   127|  Godfather The 1972|0.7990899541880941| 11|
|   408|  Close Shave A 1995|0.7945044582653329| 12|
|   169|Wrong Trousers Th...|0.7939080043840955| 13|
|   480|North by Northwes...| 0.780662935509566| 14|
|   285|  Secrets  Lies 1996|0.7804152201603161| 15|
|   114|Wallace  Gromit T...|0.780102808274138

In [49]:
df_9_.select('ItemID', 'top10_lower').filter('rn <= 10').orderBy('ItemName', ascending=True).show()

+------+------------------+
|ItemID|       top10_lower|
+------+------------------+
|   483|0.8262807973523965|
|   357| 0.808875556038692|
|   603|0.8161443229080241|
|   318|0.8336980984637925|
|    64|0.8457526835447277|
|    98|0.8334527970235317|
|    50|0.8182013061987115|
|   427|0.8134728086457121|
|    12|0.8066550869388946|
|   479|0.8332601854479507|
+------+------------------+



In [50]:
top10_lower = df_9_.select('ItemID', 'top10_lower').filter('rn <= 10').orderBy('rn', ascending=True)

top10_lower_ = [row.ItemID for row in top10_lower.toLocalIterator()]
top10_lower_ = top10_lower_[0:10]
top10_lower_ = list(map(int, top10_lower_))
top10_lower_  

[64, 318, 98, 479, 483, 50, 603, 427, 357, 12]

In [51]:
df_10_ = df_8_.select(fsql.col("ItemID").alias("ItemID"),\
                     fsql.col("ItemName").alias("ItemName"),\
                     fsql.col("top10_average").alias("top10_average"),\
                     fsql.row_number().over(Window.partitionBy()\
                                            .orderBy(fsql.col("top10_average").desc(),\
                                                     fsql.col("ItemName").asc())).alias("rn"))
df_10_.show()

+------+--------------------+-----------------+---+
|ItemID|            ItemName|    top10_average| rn|
+------+--------------------+-----------------+---+
|  1536|  Aiqing wansui 1994|              5.0|  1|
|  1653|Entertaining Ange...|              5.0|  2|
|   814|Great Day in Harl...|              5.0|  3|
|  1201|Marlene Dietrich ...|              5.0|  4|
|  1189|    Prefontaine 1997|              5.0|  5|
|  1467|Saint of Fort Was...|              5.0|  6|
|  1500|Santa with Muscle...|              5.0|  7|
|  1599|Someone Elses Ame...|              5.0|  8|
|  1293|       Star Kid 1997|              5.0|  9|
|  1122|They Made Me a Cr...|              5.0| 10|
|  1449|Pather Panchali 1955|            4.625| 11|
|  1398|           Anna 1996|              4.5| 12|
|  1594|        Everest 1998|              4.5| 13|
|   119|Maya Lin A Strong...|              4.5| 14|
|  1642|Some Mothers Son ...|              4.5| 15|
|   408|  Close Shave A 1995|4.491071428571429| 16|
|   318|Schi

In [52]:
top10_average = df_10_.select('ItemID', 'top10_average').filter('rn <= 10').orderBy('rn', ascending=True)

top10_average_ = [row.ItemID for row in top10_average.toLocalIterator()]
top10_average_ = top10_average_[0:10]
top10_average_ = list(map(int, top10_average_))
top10_average_

[1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122]

In [53]:
df_11_ = df_8_.select(fsql.col("ItemID").alias("ItemID"),\
                     fsql.col("ItemName").alias("ItemName"),\
                     fsql.col("top10_rating").alias("top10_rating"),\
                     fsql.row_number().over(Window.partitionBy()\
                                            .orderBy(fsql.col("top10_rating").desc(),\
                                                     fsql.col("ItemName").asc())).alias("rn"))
df_11_.show()

+------+--------------------+------------------+---+
|ItemID|            ItemName|      top10_rating| rn|
+------+--------------------+------------------+---+
|   318|Schindlers List 1993| 4.331876436781609|  1|
|    64|Shawshank Redempt...| 4.307786786786787|  2|
|   483|     Casablanca 1942| 4.298610921501706|  3|
|    50|      Star Wars 1977| 4.293037914691943|  4|
|    12|Usual Suspects Th...| 4.250766561514196|  5|
|   603|    Rear Window 1954| 4.221980694980695|  6|
|    98|Silence of the La...| 4.203393181818182|  7|
|   127|  Godfather The 1972|4.2019287257019435|  8|
|   408|  Close Shave A 1995| 4.194401234567901|  9|
|   169|Wrong Trousers Th...| 4.187458333333334| 10|
|   174|Raiders of the Lo...| 4.175517021276596| 11|
|   357|One Flew Over the...| 4.170359872611464| 12|
|   313|        Titanic 1997|         4.1562325| 13|
|   427|To Kill a Mocking...| 4.150531598513011| 14|
|   134|   Citizen Kane 1941| 4.139084677419355| 15|
|   172|Empire Strikes Ba...| 4.12348441247002

In [54]:
top10_rating = df_11_.select('ItemID', 'top10_rating').filter('rn <= 10').orderBy('rn', ascending=True)

top10_rating_ = [row.ItemID for row in top10_rating.toLocalIterator()]
top10_rating_ = top10_rating_[0:10]
top10_rating_ = list(map(int, top10_rating_))
top10_rating_

[318, 64, 483, 50, 12, 603, 98, 127, 408, 169]

In [55]:
result_dict_ = {} 
result_dict_["top10_rates"] = top10_rates_
result_dict_["top10_average"] = top10_average_
result_dict_["top10_rating"] = top10_rating_
result_dict_["top10_lower"] = top10_lower_
  
print(result_dict_) 

{'top10_rates': [50, 258, 100, 181, 294, 286, 288, 1, 300, 121], 'top10_average': [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122], 'top10_rating': [318, 64, 483, 50, 12, 603, 98, 127, 408, 169], 'top10_lower': [64, 318, 98, 479, 483, 50, 603, 427, 357, 12]}


In [56]:
import json

with open('lab06s.json', 'w') as fp:
    json.dump(result_dict_, fp)